In [4]:
# =========================================================
# 02_safety_layer_dynamic.ipynb  (Final Version)
# Author: Apoorva Sharma
# Purpose: Dynamic safety filtering + model saving
# =========================================================

import pandas as pd
import numpy as np
from pathlib import Path
import json, re, pickle

# ----------------------------
# 0️⃣ Paths & Setup
# ----------------------------
BASE_DIR = Path("D:/Complete_Data/nutrition_project")
PROCESSED_DIR = BASE_DIR / "processed_data"
RESULTS_DIR = BASE_DIR / "results"
MODEL_DIR = BASE_DIR / "recommendation_system" / "models"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

INPUT_PATH = PROCESSED_DIR / "recipes_enriched.csv"
OUTPUT_GLOBAL_SAFE = PROCESSED_DIR / "recipes_safe.csv"

# ----------------------------
# 1️⃣ Load data
# ----------------------------
recipes = pd.read_csv(INPUT_PATH)
print("✅ Loaded recipes:", recipes.shape)

def normalize_text(s):
    if pd.isna(s): return ""
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

if "ingredient_text" not in recipes.columns:
    recipes["ingredient_text"] = recipes[[c for c in recipes.columns if "ingredient" in c.lower()][0]]
recipes["ingredient_text_clean"] = recipes["ingredient_text"].apply(normalize_text)

# ----------------------------
# 2️⃣ Dictionaries
# ----------------------------
ALLERGENS = {...}  # keep same from your version
DIETS = {...}
HEALTH_RESTRICTIONS = {...}
MEDICATION_RESTRICTIONS = {...}

# ----------------------------
# 3️⃣ Helper functions
# ----------------------------
def text_contains_any(text, keywords):
    text = normalize_text(text)
    for kw in keywords:
        if kw.replace("_", " ") in text:
            return True
    return False

def violates_allergen(txt, allergies):
    return any(text_contains_any(txt, ALLERGENS.get(a, [])) for a in allergies)

def violates_diet(txt, diet):
    return text_contains_any(txt, DIETS.get(diet, [])) if diet else False

def violates_conditions(txt, conditions):
    flags = {c: text_contains_any(txt, HEALTH_RESTRICTIONS.get(c, [])) for c in conditions}
    return flags

def violates_medications(txt, meds):
    flags = {m: text_contains_any(txt, MEDICATION_RESTRICTIONS.get(m, [])) for m in meds}
    return flags

# ----------------------------
# 4️⃣ Safety filter
# ----------------------------
def filter_recipes_for_user(recipes_df, user_allergies=None, user_diet=None, user_conditions=None, user_meds=None, user_id="default"):
    user_allergies, user_conditions, user_meds = user_allergies or [], user_conditions or [], user_meds or []

    df = recipes_df.copy()
    df["contains_allergen"] = df["ingredient_text_clean"].apply(lambda x: violates_allergen(x, user_allergies))
    df["violates_diet"] = df["ingredient_text_clean"].apply(lambda x: violates_diet(x, user_diet))
    
    # Health + Meds
    for cond in user_conditions:
        df[f"violates_{cond}"] = df["ingredient_text_clean"].apply(lambda x: text_contains_any(x, HEALTH_RESTRICTIONS.get(cond, [])))
    for med in user_meds:
        df[f"violates_med_{med}"] = df["ingredient_text_clean"].apply(lambda x: text_contains_any(x, MEDICATION_RESTRICTIONS.get(med, [])))
    
    # Combined
    flag_cols = [c for c in df.columns if c.startswith("violates_") or c == "contains_allergen"]
    df["is_safe_for_user"] = ~df[flag_cols].any(axis=1)
    safe_df = df[df["is_safe_for_user"]]

    # Save per user
    user_path = RESULTS_DIR / f"{user_id}_recipes_safe.csv"
    safe_df.to_csv(user_path, index=False)
    print(f"✅ {user_id}: {len(safe_df)} safe recipes saved → {user_path}")

    return safe_df, df

# ----------------------------
# 5️⃣ Example users
# ----------------------------
safe_default, _ = filter_recipes_for_user(recipes, [], None, [], [], "global_default")

# Save global safe recipes
safe_default.to_csv(OUTPUT_GLOBAL_SAFE, index=False)
print(f"✅ Global safe recipes saved to: {OUTPUT_GLOBAL_SAFE}")

# ----------------------------
# 6️⃣ Save models (rules)
# ----------------------------
safety_rules = {
    "ALLERGENS": ALLERGENS,
    "DIETS": DIETS,
    "HEALTH_RESTRICTIONS": HEALTH_RESTRICTIONS,
    "MEDICATION_RESTRICTIONS": MEDICATION_RESTRICTIONS,
}
with open(MODEL_DIR / "safety_rules.pkl", "wb") as f:
    pickle.dump(safety_rules, f)

print(f"💾 Safety rules saved → {MODEL_DIR / 'safety_rules.pkl'}")
print("✅ Safety Layer complete.")


✅ Loaded recipes: (17775, 21)
✅ global_default: 17775 safe recipes saved → D:\Complete_Data\nutrition_project\results\global_default_recipes_safe.csv
✅ Global safe recipes saved to: D:\Complete_Data\nutrition_project\processed_data\recipes_safe.csv
💾 Safety rules saved → D:\Complete_Data\nutrition_project\recommendation_system\models\safety_rules.pkl
✅ Safety Layer complete.
